In [1]:
import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
def get_new_id(teams):
    id_list = sorted(teams['team_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

In [3]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [4]:
tournaments = get_tournaments(where='L.League_Short="LCL"')
tournaments = tournaments.sort_values(by=['Year', 'DateStart', 'Date']).reset_index(drop=True)
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LCL 2016 Spring,LCL/2016 Season/Spring Season,2016-01-16,2016-02-28,LoL Continental League,CIS,Online,LCL 2016 Spring,Spring,None,Primary,0,0,1,2016,1,1
1,LCL 2016 Spring Playoffs,LCL/2016 Season/Spring Playoffs,2016-03-26,2016-04-03,LoL Continental League,CIS,Offline,LCL 2016 Spring Playoffs,Spring,None,Primary,0,1,1,2016,1,1
2,LCL 2016 Summer Promotion,LCL/2016 Season/Summer Promotion,2016-04-09,2016-04-10,LoL Continental League,CIS,None,LCL 2016 Summer Promotion,Summer,None,Primary,1,0,1,2016,1,1
3,LCL 2016 Summer,LCL/2016 Season/Summer Season,2016-05-28,2016-07-10,LoL Continental League,CIS,Online,LCL 2016 Summer,Summer,None,Primary,0,0,1,2016,1,1
4,LCL 2016 Summer Playoffs,LCL/2016 Season/Summer Playoffs,2016-08-13,2016-08-14,LoL Continental League,CIS,Offline,LCL 2016 Summer Playoffs,Summer,None,Primary,0,1,1,2016,1,1
5,LCL 2017 Spring Promotion,LCL/2017 Season/Spring Promotion,2016-07-23,2016-07-24,LoL Continental League,CIS,None,LCL 2017 Spring Promotion,Spring,None,Primary,1,0,1,2017,1,1
6,LCL 2017 Spring,LCL/2017 Season/Spring Season,2017-01-28,2017-03-20,LoL Continental League,CIS,Offline,LCL 2017 Spring,Spring,None,Primary,0,0,1,2017,1,1
7,LCL 2017 Summer Promotion,LCL/2017 Season/Summer Promotion,2017-03-25,2017-03-26,LoL Continental League,CIS,None,LCL 2017 Summer Promotion,Summer,None,Primary,1,0,1,2017,1,1
8,LCL 2017 Spring Playoffs,LCL/2017 Season/Spring Playoffs,2017-04-01,2017-04-09,LoL Continental League,CIS,Offline,LCL 2017 Spring Playoffs,Spring,None,Primary,0,1,1,2017,1,1
9,LCL 2017 Summer,LCL/2017 Season/Summer Season,2017-06-24,2017-08-06,LoL Continental League,CIS,Offline,LCL 2017 Summer,Summer,None,Primary,0,0,1,2017,1,1


In [96]:
for i in range(len(tournaments)):
    scoreboard_games = get_scoreboard_games(
        where=f'T.OverviewPage="{tournaments["OverviewPage"][i]}"'
    )
    if scoreboard_games is not None:
        scoreboard_games = scoreboard_games.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
        teams = pd.read_csv('teams_id.csv')
        names = []
        for name in team_names:
            if name not in teams['team'].values:
                names.append(name)
        if len(names) > 0:
            names = sorted(names)
            break
print(scoreboard_games['OverviewPage'].unique())
print(names)

TypeError: 'NoneType' object is not subscriptable

In [27]:
teams.loc[teams['team'].str.contains('Hard')]

,team,team_id
167,Hard Random,417


In [92]:
teams = pd.concat(
    [
        teams,
        pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
        # pd.Series({'team': names.pop(0), 'team_id': 417}).to_frame().T
    ],
    ignore_index=True
)
print(names)
teams

[]


,team,team_id
0,100 Thieves,253
1,1UP Gaming,202
2,2144 Danmu Gaming,91
3,5 Ronin,386
4,7th heaven,294
...,...,...
527,n!faculty,168
528,paiN Gaming,262
529,team propertybb2,139
530,vVv Gaming,213


In [93]:
names

[]

In [94]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

,team,team_id
0,100 Thieves,253
1,1UP Gaming,202
2,2144 Danmu Gaming,91
3,5 Ronin,386
4,7th heaven,294
...,...,...
527,mousesports,123
528,n!faculty,168
529,paiN Gaming,262
530,team propertybb2,139


In [95]:
teams.to_csv('teams_id.csv', index=False)